In [7]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("najzeko/steam-reviews-2021")

# print("Path to dataset files:", path)

# **Question 6**

I’ve decided to analyze the data in chunks of 3,000,000 rows at a time. 
- For question one, I'm creating an 'updated_review' feature for all rows to indicate whether each review has been updated. Subsequently, I calculate the number of updated reviews and total reviews for each chunk, aggregate them, and then calculate the rate of updated reviews relative to total reviews.

- For question two, I assume that each data point is collected/updated at the 'timestamp_updated' time. I extract only the most recent data point for each user in each chunk, retaining specific columns: 'author.num_games_owned', 'author.num_reviews' and 'timestamp_updated'. Finally, I combine all saved chunks by repeating the above mentioned step (selecting the latest observation for each user). I've noticed that some values in author.num_games_owned and author.num_reviews fall outside a logical range. Values up to the 90th percentile appear reasonable, the maximum values are extremely high and significantly skew the average values of these features. Therefore, I've decided to cap both features at the 90th percentile and then calculate the average number of games and reviews per reviewer.

In [1]:
import pandas as pd
import numpy as np
import gc

import os

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
chunksize = 3000000

chunk = 1

dfs = []
# Iteration over chunks
for df in pd.read_csv('C:/Users/User/.cache/kagglehub/datasets/najzeko/steam-reviews-2021/versions/1/steam_reviews.csv', chunksize=chunksize):

    # Creating Bollean variable to catch whether review is updated or not; 1 - updated, 0 - not updated
    df['updated_review'] = np.where(df['timestamp_created']!=df['timestamp_updated'],1,0)

    dfs.append(df.groupby('language')['updated_review'].agg(['sum', 'count']))

    df = df.loc[df.groupby('author.steamid')['timestamp_updated'].idxmax(), ['language','author.steamid','author.num_games_owned', 'author.num_reviews','timestamp_updated']]
    df.to_csv(f'RQ6_agg_data_chunk_{chunk}_v2.csv', index=False) # save as csv file

    chunk+=1

In [3]:
likely_upd = pd.concat(dfs).reset_index()

likely_upd = likely_upd.groupby('language').sum().reset_index()

In [4]:
likely_upd['updated_prob'] = round(likely_upd['sum'] / likely_upd['count'] *100,1) # Chance to update review

In [5]:
likely_upd.sort_values('updated_prob', ascending=False, ignore_index=True)

,language,sum,count,updated_prob
0,japanese,17111,81754,20.9
1,russian,348658,2348900,14.8
2,koreana,88348,613632,14.4
3,latam,9574,70103,13.7
4,ukrainian,2747,21169,13.0
5,turkish,82754,635868,13.0
6,italian,17209,133307,12.9
7,english,1146113,9635437,11.9
8,czech,15836,133980,11.8
9,tchinese,23747,218203,10.9


In [6]:
likely_upd[likely_upd['language'].isin(['english','spanish'])]

,language,sum,count,updated_prob
5,english,1146113,9635437,11.9
21,spanish,80804,813320,9.9


In [7]:
# The directory where the CSV files are saved
directory = 'C:/Users/User/Desktop/Sapienza/ADM/Homework 2'

# Create an empty list to store the DataFrames
dfs = []

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('v2.csv') and filename.startswith('RQ6'):
        # Read the CSV file and append it to the list of DataFrames
        file_path = os.path.join(directory, filename)
        dfs.append(pd.read_csv(file_path))

# Concatenate the DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

In [8]:
combined_df = combined_df.loc[combined_df.groupby('author.steamid')['timestamp_updated'].idxmax()]

In [19]:
gc.collect()

80

In [18]:
print('Description of  author.num_games_owned')
combined_df['author.num_games_owned'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

Description of  author.num_games_owned


count        12406560
mean          1063569
std        2162692231
min                 0
10%                 4
20%                10
30%                17
40%                26
50%                38
60%                54
70%                79
80%               119
90%               206
max     4398046511619
Name: author.num_games_owned, dtype: float64

In [17]:
print('Description of  author.num_reviews')
combined_df['author.num_reviews'].describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

Description of  author.num_reviews


count        12406560
mean           708991
std        1765830883
min                 1
10%                 1
20%                 1
30%                 1
40%                 2
50%                 2
60%                 3
70%                 3
80%                 5
90%                 8
max     4398046511108
Name: author.num_reviews, dtype: float64

In [21]:
combined_df = combined_df.reset_index(drop=True)

In [56]:
# There are outliers in English, Latam groups, but to be consistent for all groups I decided to cap these features on 90th precentile 
combined_df.groupby('language')[['author.num_games_owned', 'author.num_reviews']].max().reset_index()

,language,author.num_games_owned,author.num_reviews
0,brazilian,13302,2835
1,bulgarian,2305,240
2,czech,10286,1030
3,danish,4957,435
4,dutch,10158,183
5,english,4398046511619,4398046511105
6,finnish,12214,233
7,french,12863,1147
8,german,14988,1572
9,greek,3460,485


In [61]:
combined_df.groupby('language')[['author.num_games_owned', 'author.num_reviews']].min().reset_index()

,language,author.num_games_owned,author.num_reviews
0,brazilian,0,1
1,bulgarian,0,1
2,czech,0,1
3,danish,0,1
4,dutch,0,1
5,english,0,1
6,finnish,0,1
7,french,0,1
8,german,0,1
9,greek,0,1


In [49]:
# Caltulating percentiles for each group
quantiles = combined_df.groupby('language')[['author.num_games_owned', 'author.num_reviews']].quantile(0.9).reset_index()
quantiles.columns = ['language', '90th_quantile_num_games', '90th_quantile_num_reviews']

In [58]:
combined_df = combined_df.merge(quantiles, on='language', how='inner')

In [65]:
# Capping fetures
combined_df['transformed_num_games'] = np.where(combined_df['author.num_games_owned'] > combined_df['90th_quantile_num_games'], combined_df['90th_quantile_num_games'], combined_df['author.num_games_owned'])
combined_df['transformed_num_reviews'] = np.where(combined_df['author.num_reviews'] > combined_df['90th_quantile_num_reviews'], combined_df['90th_quantile_num_reviews'], combined_df['author.num_reviews'])

In [67]:
# I make another assumption that num_reviews cannot be more than num_games reviewer owns, I checked this condition in the data and there are few observations which break it.
(combined_df['transformed_num_games']<combined_df['transformed_num_reviews']).sum()

48890

In [68]:
# If num_reviews is more than num_games than cap with num_games
combined_df['transformed_num_reviews'] = np.where(combined_df['transformed_num_reviews'] > combined_df['transformed_num_games'], combined_df['transformed_num_games'], combined_df['transformed_num_reviews'])

In [69]:
group_df = combined_df.groupby('language')[['transformed_num_games', 'transformed_num_reviews']].mean().reset_index()

In [72]:
group_df.sort_values('transformed_num_games', ascending=False, ignore_index=True)

,language,transformed_num_games,transformed_num_reviews
0,english,90,3
1,japanese,84,3
2,german,80,3
3,latam,77,4
4,italian,67,3
5,french,66,3
6,spanish,58,3
7,russian,58,3
8,czech,54,3
9,norwegian,53,2


In [71]:
group_df[group_df['language'].isin(['english','spanish'])]

,language,transformed_num_games,transformed_num_reviews
5,english,90,3
21,spanish,58,3


In [79]:
combined_df.to_csv('RQ6_QUESTION_2_DATA.csv', index=False) # Need this data to calculate correlation

## **Results**

1. The group most likely to edit or update their reviews after submitting them is the Japanese group (20.9%), while the least likely is the Danish group (4.6%). Between English and Spanish reviewers, the English group is more prone to edit or update their reviews, with a likelihood of 11.9%, compared to a 9.9% likelihood for the Spanish group.

2. - **Average # of Games** - English speakers own the most games with 90 games on average. Spanish speakers show good engagement with 58 games, which is above average.
Japanese and German users follow English as top owners with 84 and 80 games. Romanian users own the least games at 23, followed by Vietnamese users at 32 games.
Most language groups own between 40 to 60 games. There is a big gap between the highest (English - 90) and lowest (Romanian - 23) game counts.

    - **Average # of Reviews** - The review patterns are more consistent across languages. Most groups, including English and Spanish, write 3 reviews on average.
Latam stands out as the only group with 4 reviews. Several languages including Bulgarian, Danish, and Dutch show lower engagement with 2 reviews.

    - English speakers own more games but write the same number of reviews as Spanish speakers. While English users have 90 games, Spanish users have 58 games. Both groups write 3 reviews on average. This suggests that owning more games doesn't necessarily lead to writing more reviews.

# **Question 7**

I’ve decided to analyze the data in chunks of 1,000,000 rows at a time.
- For the first and second questions, analyzing the data chunk by chunk and aggregating the information was straightforward. I am calculating the number of total reviews and reviews that received at least one helpful vote for each recommendation type. Then, I'm calculating the ratio between reviews with helpful votes and total reviews for two groups: the entire population and the population that did not recommend the game.

- For the third question to identify whether event $A$ and $B$ are independet I need to calculate their marginal probabilities and probability of $A∩B$. If the intersection is factorized $P(A∩B)=P(A) * P(B)$ then events are independent. We have already computed $P(A)$ (a review receiving at least one helpful vote), we need to calculate $P(B)$ (the reviewer has submitted at least five reviews before the current review) and $P(A∩B)$. The way I'm computing $P(B)$ is that, I'm ranking each users reviews based on timestamp_created, where first review has rank -> 0 and the n-th has rank -> n-1. Afterward I'm only counting datapoints, where rank is at least 5 and devide it to total number of observations. Finally, I compute $P(A∩B)$ ann check independence condition.

- To answer question 4, I decided to calculate correlation coefficient which will give us statistical proof whether our hypotesis is correct or not

In [84]:
import pandas as pd
import numpy as np
import gc

import os

pd.set_option('display.max_columns', None)

In [2]:
chunksize = 3000000
chunk = 1

helpful_votes = []


for df in pd.read_csv('C:/Users/User/.cache/kagglehub/datasets/najzeko/steam-reviews-2021/versions/1/steam_reviews.csv', chunksize=chunksize):

    # Creating flag, whether reviewe has at least one "helpful" reaction or not; 1 - it has, 0 - it hasn't
    df['helpful_vote_flg'] = np.where(df['votes_helpful']>0, 1, 0)

    # Grouping chunk based on recommended flag and calculating total reviews and number of helpful voted
    grouped_df = df.groupby('recommended')['helpful_vote_flg'].agg(['sum', 'count']).reset_index().rename(columns={'sum':'helpful_votes', 'count': 'total_reviews'})

    # Saving necessary columns for the third and forth question 
    df[['author.steamid','timestamp_created','helpful_vote_flg','author.num_games_owned','author.num_reviews']].to_csv(f'RQ7_agg_data_chunk_{chunk}.csv', index=False)

    helpful_votes.append(grouped_df)

    chunk += 1

In [3]:
df_votes = pd.concat(helpful_votes, ignore_index=True)

df_votes = df_votes.groupby('recommended').sum().reset_index()

In [4]:
# the probability of submitting a review and receiving at least one helpful vote from other users 
round(df_votes['helpful_votes'].sum() / df_votes['total_reviews'].sum(),2)

0.28

In [5]:
# the probability of submitting a review and receiving at least one helpful vote from other users, given that you don’t recommend the app
round(df_votes.loc[df_votes['recommended']==False,'helpful_votes'][0] / df_votes.loc[df_votes['recommended']==False,'total_reviews'][0],2)

0.48

In [2]:
# The directory where the CSV files are saved
directory = 'C:/Users/User/Desktop/Sapienza/ADM/Homework 2'

# Create an empty list to store the DataFrames
dfs = []

# Loop through all the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv') and filename.startswith('RQ7'):
        # Read the CSV file and append it to the list of DataFrames
        file_path = os.path.join(directory, filename)
        dfs.append(pd.read_csv(file_path))

# Concatenate the DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

In [7]:
# ranking reviews based on timestamp_created, to identify whether costumer had at least 5 reviews before the current review or not
combined_df['rank_reviews'] = combined_df.groupby('author.steamid')['timestamp_created'].rank('first')-1

In [11]:
total_reviews = len(combined_df)

In [44]:
# the probability that “the reviewer has submitted at least five reviews before the current review”
round((combined_df['rank_reviews']>=5).sum() / total_reviews,2)

0.08

In [45]:
# the probability that “the reviewer has submitted at least five reviews before the current review” and "receiving at least one helpful vote from other users"
round(((combined_df['rank_reviews']>=5) & (combined_df['helpful_vote_flg']>0)).sum() / total_reviews,2)

0.03

In [48]:
# the probability of submitting a review and receiving at least one helpful vote from other users -> P(A)=0.28
# the probability that “the reviewer has submitted at least five reviews before the current review” -> P(B)=0.08
# the probability that “the reviewer has submitted at least five reviews before the current review” and "receiving at least one helpful vote from other users" -> P(A∩B) = 0.03
# If events A and B are indepented -> P(A∩B)=P(A) * P(B)

round(0.28 * 0.08,2) # They aren't independet !!! Logically if a user has previous reviews there should be more chance to get helpful votes than for the first time reviewer.

0.02

In [43]:
# Calculate correlation between number of games owned by reviewer and number of reviews written by reviewer to check hypothesis

combined_df[['author.num_games_owned', 'author.num_reviews']].corr().iloc[0,1]

# As we see from the result there is almost no correlation between those features, which means that there is no any relationship. Therefore hyphotesis is wrong !!!

-1.454229766484716e-07

In [80]:
# To calculate correlation between number of reviews and number of games, I'm using RQ6 question 2 data, that I prepared previously.
combined_df = pd.read_csv('RQ6_QUESTION_2_DATA.csv')

In [83]:
print('Correlation between number of reviews and number of games is', round(combined_df[['transformed_num_games', 'transformed_num_reviews']].corr().iloc[0,1],2))

Correlation between number of reviews and number of games is 0.45


The correlation coefficient of 0.45 between number of games owned and reviews written refutes the original hypothesis. Instead of showing that people with more games write fewer reviews, the positive correlation indicates that users who own more games tend to write slightly more reviews. While this relationship is moderate (not strong), it suggests that engaged gamers who purchase more games are also more likely to participate in reviewing them. This could be because these users are more invested in the gaming community or simply have more games to review, but the 0.45 correlation shows the relationship isn't strong enough to make definitive predictions about review behavior based solely on game ownership.

**Answer of the last question:**

Data Visualization Types and Their Uses:

1. Histograms
- Show distribution of continuous numerical data
- Help identify patterns, skewness, and outliers
- Best for: seeing how values are spread across ranges (like review scores or game prices)


2. Bar Plots
- Compare categorical data or groups
- Show differences between distinct categories
- Best for: comparing metrics across languages or recommendation types


3. Scatterplots
- Show relationships between two numerical variables
- Help identify correlations and patterns
- Best for: analyzing relationships (like games owned vs reviews written)


4. Pie Charts
- Show parts of a whole
- Represent proportions of categories
- Best for: showing market share or percentage breakdowns

How to Validate LLM Suggestions:


1. Cross-reference Sources:
- Check statistical textbooks
- Verify with data visualization guides
- Compare with academic resources

2. Practical Validation:
- Try different chart types with your data
- Compare which visualization reveals insights better
- Test if the suggested visualization answers your research question


3. Expert Review:
- Consult with data visualization experts
- Share with peers for feedback
- Check if conclusions match statistical best practices

4. Critical Assessment:
- Consider if the visualization might mislead
- Check if it's the simplest way to show the data
- Ensure it matches your audience's needs

To improve confidence in LLM suggestions:

- Always verify with multiple sources
- Test suggestions with real data
- Use established statistical guidelines
- Consider the context and audience
- Document your validation process